In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sys
from skimage.io import imread
from matplotlib import pyplot as plt
import os
# os.environ['KERAS_BACKEND'] = 'theano'
from keras import models
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
path = 'Data/'
img_w = 256
img_h = 256
n_labels = 2
n_train = 6
n_test = 3

In [3]:
def label_map(labels):
    label_map = np.zeros([img_h, img_w, n_labels])    
    for r in range(img_h):
        for c in range(img_w):
            label_map[r, c, labels[r][c]] = 1
    return label_map

In [4]:
def prep_data(mode):
    assert mode in {'test', 'train'}, \
        'mode should be either \'test\' or \'train\''
    data = []
    label = []
    df = pd.read_csv(path + mode + '.csv')
    n = n_train if mode == 'train' else n_test
    for i, item in df.iterrows():
        if i >= n:
            break
        img, gt = [imread(path + item[0])], np.clip(imread(path + item[1]), 0, 1)
        data.append(img)
        label.append(label_map(gt))
        sys.stdout.write('\r')
        sys.stdout.write(mode + ": [%-20s] %d%%" % ('=' * int(20. * (i + 1) / n - 1) + '>',
                                                    int(100. * (i + 1) / n)))
        sys.stdout.flush()
    sys.stdout.write('\r')
    sys.stdout.flush()
    data, label = np.array(data), np.array(label).reshape((n, img_h * img_w, n_labels))

    print(mode + ': OK')
    print('\tshapes: {}, {}'.format(data.shape, label.shape))
    print('\ttypes:  {}, {}'.format(data.dtype, label.dtype))
    print('\tmemory: {}, {} MB'.format(data.nbytes / 1048576, label.nbytes / 1048576))
    
    data = data.transpose((0,2,3,1))
    return data, label

In [5]:
def plot_results(output):
    gt = []
    df = pd.read_csv(path + 'test.csv')
    for i, item in df.iterrows():
        gt.append(np.clip(imread(path + item[1]), 0, 1))

    plt.figure(figsize=(15, 2 * n_test))
    for i, item in df.iterrows():
        plt.subplot(n_test, 4, 4 * i + 1)
        plt.title('Ground Truth')
        plt.axis('off')
        gt = imread(path + item[1])
        plt.imshow(np.clip(gt, 0, 1))

        plt.subplot(n_test, 4, 4 * i + 2)
        plt.title('Prediction')
        plt.axis('off')
        labeled = np.argmax(output[i], axis=-1)
        plt.imshow(labeled)

        plt.subplot(n_test, 4, 4 * i + 3)
        plt.title('Heat map')
        plt.axis('off')
        plt.imshow(output[i][:, :, 1])

        plt.subplot(n_test, 4, 4 * i + 4)
        plt.title('Comparison')
        plt.axis('off')
        rgb = np.empty((img_h, img_w, 3))
        rgb[:, :, 0] = labeled
        rgb[:, :, 1] = imread(path + item[0])
        rgb[:, :, 2] = gt
        plt.imshow(rgb)

    plt.savefig('result.png')
    plt.show()

In [6]:
with open('model_5l.json') as model_file:
    autoencoder = models.model_from_json(model_file.read())
    

In [7]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
__________

In [8]:
optimizer = SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=False)
autoencoder.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
print('Compiled: OK')


Compiled: OK


In [9]:
## loading  dataset
train_data, train_label = prep_data('train')


train: OK
	shapes: (6, 1, 256, 256), (6, 65536, 2)
	types:  uint8, float64
	memory: 0.375, 6.0 MB


In [10]:
from keras.callbacks import ModelCheckpoint
# Save check point
filepath = "segnet.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only = True, mode ='max')
callbacks_list= [checkpoint]

nb_epoch = 50
batch_size = 18 

history = autoencoder.fit(train_data, train_label, batch_size=batch_size, nb_epoch=nb_epoch, verbose=2)


    


C:\Users\hoanganh\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
 - 5s - loss: 0.9080 - acc: 0.4768
Epoch 2/50
 - 0s - loss: 0.8564 - acc: 0.5017
Epoch 3/50
 - 0s - loss: 0.8291 - acc: 0.5177
Epoch 4/50
 - 0s - loss: 0.7823 - acc: 0.5405
Epoch 5/50
 - 0s - loss: 0.7478 - acc: 0.5586
Epoch 6/50
 - 0s - loss: 0.6964 - acc: 0.5938
Epoch 7/50
 - 0s - loss: 0.6678 - acc: 0.6142
Epoch 8/50
 - 0s - loss: 0.6313 - acc: 0.6499
Epoch 9/50
 - 0s - loss: 0.6018 - acc: 0.6771
Epoch 10/50
 - 0s - loss: 0.5932 - acc: 0.6865
Epoch 11/50
 - 0s - loss: 0.5625 - acc: 0.7109
Epoch 12/50
 - 0s - loss: 0.5324 - acc: 0.7380
Epoch 13/50
 - 0s - loss: 0.5187 - acc: 0.7489
Epoch 14/50
 - 0s - loss: 0.4871 - acc: 0.7781
Epoch 15/50
 - 0s - loss: 0.4604 - acc: 0.8025
Epoch 16/50
 - 0s - loss: 0.4417 - acc: 0.8183
Epoch 17/50
 - 0s - loss: 0.4347 - acc: 0.8233
Epoch 18/50
 - 0s - loss: 0.4250 - acc: 0.8302
Epoch 19/50
 - 0s - loss: 0.3945 - acc: 0.8556
Epoch 20/50
 - 0s - loss: 0.3707 - acc: 0.8757
Epoch 21/50
 - 0s - loss: 0.3603 - acc: 0.8814
Epoch 22/50
 - 0s - lo

In [11]:
autoencoder.load_weights('model_5l_weight_ep50.hdf5')

OSError: Unable to open file (file signature not found)

In [ ]:
# from keras.utils.visualize_util import plot
# plot(autoencoder, to_file='model.png',autoencoder)

from keras.utils import plot_model
plot_model(autoencoder, to_file='model.png')



In [ ]:
test_data, test_label = prep_data('test')

In [ ]:
score = autoencoder.evaluate(test_data, test_label, verbose=0)
print('Test score:', score[0])

print('Test accuracy:', score[1])

In [ ]:
output = autoencoder.predict_proba(test_data, verbose=0)
output = output.reshape((output.shape[0], img_h, img_w, n_labels))

plot_results(output)